# Task 2: Model Building and Training

This notebook implements:
1. Data Preparation
2. Baseline Model (Logistic Regression)
3. Ensemble Models (XGBoost, Random Forest, LightGBM)
4. Cross-Validation
5. Model Comparison and Selection


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn imports
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score, confusion_matrix,
    classification_report, roc_curve, precision_recall_curve
)

# XGBoost and LightGBM
import xgboost as xgb
import lightgbm as lgb

# Model persistence
import joblib

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")


## 1. Data Preparation

Load the processed training and test datasets that were prepared in the feature engineering step.


In [ ]:
# Load processed data
data_dir = Path('../data/processed')

print("Loading processed datasets...")
X_train = pd.read_csv(data_dir / 'X_train_processed.csv')
y_train = pd.read_csv(data_dir / 'y_train_processed.csv')
X_test = pd.read_csv(data_dir / 'X_test_processed.csv')
y_test = pd.read_csv(data_dir / 'y_test_processed.csv')

# Extract target variable (handle both single column and multi-column cases)
if y_train.shape[1] == 1:
    y_train = y_train.iloc[:, 0]
else:
    y_train = y_train['class']

if y_test.shape[1] == 1:
    y_test = y_test.iloc[:, 0]
else:
    y_test = y_test['class']

print(f"\nTraining set shape: X_train={X_train.shape}, y_train={y_train.shape}")
print(f"Test set shape: X_test={X_test.shape}, y_test={y_test.shape}")

# Check class distribution
print(f"\nTraining set class distribution:")
print(y_train.value_counts())
print(f"\nTraining set class percentages:")
print(y_train.value_counts(normalize=True) * 100)

print(f"\nTest set class distribution:")
print(y_test.value_counts())
print(f"\nTest set class percentages:")
print(y_test.value_counts(normalize=True) * 100)

# Verify no missing values
print(f"\nMissing values in X_train: {X_train.isnull().sum().sum()}")
print(f"Missing values in X_test: {X_test.isnull().sum().sum()}")
print(f"Missing values in y_train: {y_train.isnull().sum()}")
print(f"Missing values in y_test: {y_test.isnull().sum()}")


## 2. Evaluation Metrics Function

Define a function to evaluate models using appropriate metrics for imbalanced data.


In [ ]:
def evaluate_model(y_true, y_pred, y_pred_proba=None, model_name="Model"):
    """
    Evaluate model performance with multiple metrics suitable for imbalanced data.
    
    Parameters:
    -----------
    y_true : array-like
        True labels
    y_pred : array-like
        Predicted labels
    y_pred_proba : array-like, optional
        Predicted probabilities for positive class
    model_name : str
        Name of the model for display
    
    Returns:
    --------
    dict : Dictionary containing all metrics
    """
    metrics = {}
    
    # Basic metrics
    metrics['accuracy'] = accuracy_score(y_true, y_pred)
    metrics['precision'] = precision_score(y_true, y_pred, zero_division=0)
    metrics['recall'] = recall_score(y_true, y_pred, zero_division=0)
    metrics['f1_score'] = f1_score(y_true, y_pred, zero_division=0)
    
    # Probability-based metrics (if probabilities provided)
    if y_pred_proba is not None:
        metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba)
        metrics['pr_auc'] = average_precision_score(y_true, y_pred_proba)
    
    # Print results
    print(f"\n{'='*60}")
    print(f"{model_name} - Evaluation Metrics")
    print(f"{'='*60}")
    print(f"Accuracy:  {metrics['accuracy']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall:    {metrics['recall']:.4f}")
    print(f"F1-Score:  {metrics['f1_score']:.4f}")
    if y_pred_proba is not None:
        print(f"ROC-AUC:   {metrics['roc_auc']:.4f}")
        print(f"PR-AUC:    {metrics['pr_auc']:.4f}")
    print(f"{'='*60}\n")
    
    return metrics

def plot_confusion_matrix(y_true, y_pred, model_name="Model"):
    """Plot confusion matrix"""
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Legitimate', 'Fraud'],
                yticklabels=['Legitimate', 'Fraud'])
    plt.title(f'{model_name} - Confusion Matrix', fontsize=14, fontweight='bold')
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.tight_layout()
    plt.show()
    
    return cm

def plot_roc_pr_curves(y_true, y_pred_proba, model_name="Model"):
    """Plot ROC and Precision-Recall curves"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # ROC Curve
    fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
    roc_auc = roc_auc_score(y_true, y_pred_proba)
    
    axes[0].plot(fpr, tpr, color='darkorange', lw=2, 
                label=f'ROC curve (AUC = {roc_auc:.4f})')
    axes[0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
    axes[0].set_xlim([0.0, 1.0])
    axes[0].set_ylim([0.0, 1.05])
    axes[0].set_xlabel('False Positive Rate', fontsize=12)
    axes[0].set_ylabel('True Positive Rate', fontsize=12)
    axes[0].set_title(f'{model_name} - ROC Curve', fontsize=14, fontweight='bold')
    axes[0].legend(loc="lower right")
    axes[0].grid(True, alpha=0.3)
    
    # Precision-Recall Curve
    precision, recall, _ = precision_recall_curve(y_true, y_pred_proba)
    pr_auc = average_precision_score(y_true, y_pred_proba)
    
    axes[1].plot(recall, precision, color='darkorange', lw=2,
                label=f'PR curve (AUC = {pr_auc:.4f})')
    axes[1].set_xlim([0.0, 1.0])
    axes[1].set_ylim([0.0, 1.05])
    axes[1].set_xlabel('Recall', fontsize=12)
    axes[1].set_ylabel('Precision', fontsize=12)
    axes[1].set_title(f'{model_name} - Precision-Recall Curve', fontsize=14, fontweight='bold')
    axes[1].legend(loc="lower left")
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

print("Evaluation functions defined successfully!")


In [ ]:
# Train Logistic Regression baseline model
print("Training Logistic Regression baseline model...")
print("="*60)

# Use class_weight='balanced' to handle imbalance
lr_model = LogisticRegression(
    random_state=42,
    max_iter=1000,
    class_weight='balanced',  # Automatically adjust class weights
    solver='lbfgs'  # Good for small to medium datasets
)

# Train the model
lr_model.fit(X_train, y_train)

# Make predictions
y_pred_lr = lr_model.predict(X_test)
y_pred_proba_lr = lr_model.predict_proba(X_test)[:, 1]

# Evaluate
lr_metrics = evaluate_model(y_test, y_pred_lr, y_pred_proba_lr, "Logistic Regression")
lr_cm = plot_confusion_matrix(y_test, y_pred_lr, "Logistic Regression")
plot_roc_pr_curves(y_test, y_pred_proba_lr, "Logistic Regression")

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_lr, target_names=['Legitimate', 'Fraud']))


## 4. Ensemble Models

Train ensemble models: Random Forest, XGBoost, and LightGBM.


### 4.1 Random Forest Classifier


In [ ]:
# Train Random Forest model
print("Training Random Forest model...")
print("="*60)

rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=5,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    verbose=0
)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf_model.predict(X_test)
y_pred_proba_rf = rf_model.predict_proba(X_test)[:, 1]

# Evaluate
rf_metrics = evaluate_model(y_test, y_pred_rf, y_pred_proba_rf, "Random Forest")
rf_cm = plot_confusion_matrix(y_test, y_pred_rf, "Random Forest")
plot_roc_pr_curves(y_test, y_pred_proba_rf, "Random Forest")

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf, target_names=['Legitimate', 'Fraud']))


### 4.2 XGBoost Classifier


In [ ]:
# Train XGBoost model
print("Training XGBoost model...")
print("="*60)

xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=len(y_train[y_train==0]) / len(y_train[y_train==1]),  # Handle imbalance
    random_state=42,
    eval_metric='logloss',
    n_jobs=-1,
    verbosity=0
)

# Train the model
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred_xgb = xgb_model.predict(X_test)
y_pred_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]

# Evaluate
xgb_metrics = evaluate_model(y_test, y_pred_xgb, y_pred_proba_xgb, "XGBoost")
xgb_cm = plot_confusion_matrix(y_test, y_pred_xgb, "XGBoost")
plot_roc_pr_curves(y_test, y_pred_proba_xgb, "XGBoost")

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_xgb, target_names=['Legitimate', 'Fraud']))


### 4.3 LightGBM Classifier


In [ ]:
# Train LightGBM model
print("Training LightGBM model...")
print("="*60)

lgb_model = lgb.LGBMClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

# Train the model
lgb_model.fit(X_train, y_train)

# Make predictions
y_pred_lgb = lgb_model.predict(X_test)
y_pred_proba_lgb = lgb_model.predict_proba(X_test)[:, 1]

# Evaluate
lgb_metrics = evaluate_model(y_test, y_pred_lgb, y_pred_proba_lgb, "LightGBM")
lgb_cm = plot_confusion_matrix(y_test, y_pred_lgb, "LightGBM")
plot_roc_pr_curves(y_test, y_pred_proba_lgb, "LightGBM")

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_lgb, target_names=['Legitimate', 'Fraud']))


## 5. Hyperparameter Tuning

Perform basic hyperparameter tuning for the best performing ensemble model.


In [ ]:
from sklearn.model_selection import GridSearchCV

# We'll tune XGBoost as it typically performs well for fraud detection
print("Performing hyperparameter tuning for XGBoost...")
print("="*60)

# Define parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.05, 0.1],
    'subsample': [0.8, 0.9]
}

# Use PR-AUC as scoring metric (better for imbalanced data)
xgb_tuned = GridSearchCV(
    estimator=xgb.XGBClassifier(
        scale_pos_weight=len(y_train[y_train==0]) / len(y_train[y_train==1]),
        random_state=42,
        eval_metric='logloss',
        n_jobs=-1,
        verbosity=0
    ),
    param_grid=param_grid,
    scoring='average_precision',  # PR-AUC
    cv=3,  # 3-fold CV for faster tuning
    n_jobs=-1,
    verbose=1
)

# Fit the model
print("Fitting GridSearchCV (this may take a few minutes)...")
xgb_tuned.fit(X_train, y_train)

print(f"\nBest parameters: {xgb_tuned.best_params_}")
print(f"Best CV score (PR-AUC): {xgb_tuned.best_score_:.4f}")

# Make predictions with tuned model
y_pred_xgb_tuned = xgb_tuned.predict(X_test)
y_pred_proba_xgb_tuned = xgb_tuned.predict_proba(X_test)[:, 1]

# Evaluate tuned model
xgb_tuned_metrics = evaluate_model(y_test, y_pred_xgb_tuned, y_pred_proba_xgb_tuned, "XGBoost (Tuned)")
xgb_tuned_cm = plot_confusion_matrix(y_test, y_pred_xgb_tuned, "XGBoost (Tuned)")
plot_roc_pr_curves(y_test, y_pred_proba_xgb_tuned, "XGBoost (Tuned)")

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_xgb_tuned, target_names=['Legitimate', 'Fraud']))


In [ ]:
# Define cross-validation function
def perform_cross_validation(model, X, y, model_name="Model", cv_folds=5):
    """
    Perform stratified k-fold cross-validation and report mean and std of metrics.
    """
    skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
    
    # Store results for each fold
    cv_results = {
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1_score': [],
        'roc_auc': [],
        'pr_auc': []
    }
    
    print(f"\n{'='*60}")
    print(f"{model_name} - {cv_folds}-Fold Cross-Validation Results")
    print(f"{'='*60}\n")
    
    fold = 1
    for train_idx, val_idx in skf.split(X, y):
        X_fold_train, X_fold_val = X.iloc[train_idx], X.iloc[val_idx]
        y_fold_train, y_fold_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # Train model
        model.fit(X_fold_train, y_fold_train)
        
        # Predictions
        y_pred = model.predict(X_fold_val)
        y_pred_proba = model.predict_proba(X_fold_val)[:, 1]
        
        # Calculate metrics
        cv_results['accuracy'].append(accuracy_score(y_fold_val, y_pred))
        cv_results['precision'].append(precision_score(y_fold_val, y_pred, zero_division=0))
        cv_results['recall'].append(recall_score(y_fold_val, y_pred, zero_division=0))
        cv_results['f1_score'].append(f1_score(y_fold_val, y_pred, zero_division=0))
        cv_results['roc_auc'].append(roc_auc_score(y_fold_val, y_pred_proba))
        cv_results['pr_auc'].append(average_precision_score(y_fold_val, y_pred_proba))
        
        print(f"Fold {fold}: PR-AUC = {cv_results['pr_auc'][-1]:.4f}, "
              f"F1 = {cv_results['f1_score'][-1]:.4f}, "
              f"Recall = {cv_results['recall'][-1]:.4f}")
        fold += 1
    
    # Calculate mean and std
    print(f"\n{'='*60}")
    print("Cross-Validation Summary (Mean ± Std)")
    print(f"{'='*60}")
    print(f"Accuracy:  {np.mean(cv_results['accuracy']):.4f} ± {np.std(cv_results['accuracy']):.4f}")
    print(f"Precision: {np.mean(cv_results['precision']):.4f} ± {np.std(cv_results['precision']):.4f}")
    print(f"Recall:    {np.mean(cv_results['recall']):.4f} ± {np.std(cv_results['recall']):.4f}")
    print(f"F1-Score:  {np.mean(cv_results['f1_score']):.4f} ± {np.std(cv_results['f1_score']):.4f}")
    print(f"ROC-AUC:   {np.mean(cv_results['roc_auc']):.4f} ± {np.std(cv_results['roc_auc']):.4f}")
    print(f"PR-AUC:    {np.mean(cv_results['pr_auc']):.4f} ± {np.std(cv_results['pr_auc']):.4f}")
    print(f"{'='*60}\n")
    
    return cv_results

# Perform cross-validation for all models
print("\n" + "="*60)
print("CROSS-VALIDATION FOR ALL MODELS")
print("="*60)

# Logistic Regression CV
lr_cv = perform_cross_validation(
    LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced', solver='lbfgs'),
    X_train, y_train, "Logistic Regression", cv_folds=5
)

# Random Forest CV
rf_cv = perform_cross_validation(
    RandomForestClassifier(n_estimators=100, max_depth=15, min_samples_split=10,
                          min_samples_leaf=5, class_weight='balanced', random_state=42, n_jobs=-1),
    X_train, y_train, "Random Forest", cv_folds=5
)

# XGBoost CV
xgb_cv = perform_cross_validation(
    xgb.XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1,
                     scale_pos_weight=len(y_train[y_train==0]) / len(y_train[y_train==1]),
                     random_state=42, eval_metric='logloss', n_jobs=-1, verbosity=0),
    X_train, y_train, "XGBoost", cv_folds=5
)

# LightGBM CV
lgb_cv = perform_cross_validation(
    lgb.LGBMClassifier(n_estimators=100, max_depth=6, learning_rate=0.1,
                     class_weight='balanced', random_state=42, n_jobs=-1, verbose=-1),
    X_train, y_train, "LightGBM", cv_folds=5
)


In [ ]:
# Create comparison DataFrame
comparison_data = {
    'Model': ['Logistic Regression', 'Random Forest', 'XGBoost', 'LightGBM', 'XGBoost (Tuned)'],
    'PR-AUC (Test)': [
        lr_metrics['pr_auc'],
        rf_metrics['pr_auc'],
        xgb_metrics['pr_auc'],
        lgb_metrics['pr_auc'],
        xgb_tuned_metrics['pr_auc']
    ],
    'F1-Score (Test)': [
        lr_metrics['f1_score'],
        rf_metrics['f1_score'],
        xgb_metrics['f1_score'],
        lgb_metrics['f1_score'],
        xgb_tuned_metrics['f1_score']
    ],
    'Recall (Test)': [
        lr_metrics['recall'],
        rf_metrics['recall'],
        xgb_metrics['recall'],
        lgb_metrics['recall'],
        xgb_tuned_metrics['recall']
    ],
    'Precision (Test)': [
        lr_metrics['precision'],
        rf_metrics['precision'],
        xgb_metrics['precision'],
        lgb_metrics['precision'],
        xgb_tuned_metrics['precision']
    ],
    'PR-AUC (CV Mean)': [
        np.mean(lr_cv['pr_auc']),
        np.mean(rf_cv['pr_auc']),
        np.mean(xgb_cv['pr_auc']),
        np.mean(lgb_cv['pr_auc']),
        xgb_tuned.best_score_ if 'xgb_tuned' in locals() else np.mean(xgb_cv['pr_auc'])
    ],
    'PR-AUC (CV Std)': [
        np.std(lr_cv['pr_auc']),
        np.std(rf_cv['pr_auc']),
        np.std(xgb_cv['pr_auc']),
        np.std(lgb_cv['pr_auc']),
        0  # GridSearchCV doesn't provide std directly
    ]
}

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('PR-AUC (Test)', ascending=False)

print("\n" + "="*80)
print("MODEL COMPARISON SUMMARY")
print("="*80)
print(comparison_df.to_string(index=False))
print("="*80)

# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# PR-AUC comparison
axes[0, 0].barh(comparison_df['Model'], comparison_df['PR-AUC (Test)'], color='steelblue')
axes[0, 0].set_xlabel('PR-AUC Score', fontsize=12)
axes[0, 0].set_title('PR-AUC Comparison (Test Set)', fontsize=14, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3, axis='x')

# F1-Score comparison
axes[0, 1].barh(comparison_df['Model'], comparison_df['F1-Score (Test)'], color='darkgreen')
axes[0, 1].set_xlabel('F1-Score', fontsize=12)
axes[0, 1].set_title('F1-Score Comparison (Test Set)', fontsize=14, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3, axis='x')

# Recall comparison
axes[1, 0].barh(comparison_df['Model'], comparison_df['Recall (Test)'], color='crimson')
axes[1, 0].set_xlabel('Recall', fontsize=12)
axes[1, 0].set_title('Recall Comparison (Test Set)', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3, axis='x')

# Precision comparison
axes[1, 1].barh(comparison_df['Model'], comparison_df['Precision (Test)'], color='orange')
axes[1, 1].set_xlabel('Precision', fontsize=12)
axes[1, 1].set_title('Precision Comparison (Test Set)', fontsize=14, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

# Cross-validation comparison
fig, ax = plt.subplots(figsize=(10, 6))
x_pos = np.arange(len(comparison_df))
width = 0.35

cv_means = comparison_df['PR-AUC (CV Mean)'].values
cv_stds = comparison_df['PR-AUC (CV Std)'].values

ax.barh(x_pos, cv_means, width, xerr=cv_stds, label='CV Mean ± Std', color='steelblue', alpha=0.7)
ax.set_yticks(x_pos)
ax.set_yticklabels(comparison_df['Model'])
ax.set_xlabel('PR-AUC Score', fontsize=12)
ax.set_title('Cross-Validation PR-AUC Comparison (Mean ± Std)', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()


In [ ]:
# Select best model based on PR-AUC (most important metric for imbalanced data)
best_model_name = comparison_df.iloc[0]['Model']
best_pr_auc = comparison_df.iloc[0]['PR-AUC (Test)']

print("\n" + "="*80)
print("MODEL SELECTION DECISION")
print("="*80)
print(f"\nSelected Best Model: {best_model_name}")
print(f"Test Set PR-AUC: {best_pr_auc:.4f}")
print(f"Test Set F1-Score: {comparison_df.iloc[0]['F1-Score (Test)']:.4f}")
print(f"Test Set Recall: {comparison_df.iloc[0]['Recall (Test)']:.4f}")
print(f"Test Set Precision: {comparison_df.iloc[0]['Precision (Test)']:.4f}")

print("\n" + "="*80)
print("JUSTIFICATION")
print("="*80)
print("""
Selection Criteria:
1. PR-AUC (Primary): Most important metric for imbalanced data, measures 
   precision-recall trade-off across all thresholds
2. F1-Score: Harmonic mean of precision and recall, balances both metrics
3. Recall: Critical for fraud detection - we want to catch as many fraud cases as possible
4. Cross-Validation Stability: Low standard deviation indicates consistent performance
5. Interpretability: Important for business stakeholders and regulatory compliance

Rationale:
- PR-AUC is preferred over ROC-AUC for imbalanced datasets because it focuses on 
  the positive class (fraud) which is our primary concern
- High recall is crucial to minimize false negatives (missed fraud cases)
- Model should show consistent performance across cross-validation folds
- Balance between performance and interpretability for business use
""")

# Get the actual best model object
if 'XGBoost (Tuned)' in best_model_name:
    best_model = xgb_tuned.best_estimator_
elif 'XGBoost' in best_model_name and 'Tuned' not in best_model_name:
    best_model = xgb_model
elif 'Random Forest' in best_model_name:
    best_model = rf_model
elif 'LightGBM' in best_model_name:
    best_model = lgb_model
else:
    best_model = lr_model

print(f"\nBest model object: {type(best_model).__name__}")
print("="*80)


## 8. Save Best Model

Save the selected best model and preprocessing objects for deployment.


In [ ]:
# Create models directory if it doesn't exist
models_dir = Path('../models')
models_dir.mkdir(exist_ok=True)

# Save best model
model_filename = models_dir / f'best_model_{best_model_name.lower().replace(" ", "_").replace("(", "").replace(")", "")}.pkl'
joblib.dump(best_model, model_filename)
print(f"Best model saved to: {model_filename}")

# Also save all models for comparison
joblib.dump(lr_model, models_dir / 'logistic_regression.pkl')
joblib.dump(rf_model, models_dir / 'random_forest.pkl')
joblib.dump(xgb_model, models_dir / 'xgboost.pkl')
joblib.dump(lgb_model, models_dir / 'lightgbm.pkl')
if 'xgb_tuned' in locals():
    joblib.dump(xgb_tuned.best_estimator_, models_dir / 'xgboost_tuned.pkl')

print("\nAll models saved successfully!")

# Save comparison results
comparison_df.to_csv(models_dir / 'model_comparison_results.csv', index=False)
print(f"Model comparison results saved to: {models_dir / 'model_comparison_results.csv'}")

print("\n" + "="*80)
print("TASK 2 COMPLETED SUCCESSFULLY!")
print("="*80)
print(f"\nBest Model: {best_model_name}")
print(f"Model saved to: {model_filename}")
print(f"\nNext Steps:")
print("1. Proceed to Task 3: Model Explainability using SHAP")
print("2. Analyze feature importance from the best model")
print("3. Generate explainability visualizations")
print("="*80)
